In [1]:
import arcpy
import pyowm
import csv

In [26]:
def rainupdate(fold_csv,root):
    owm = pyowm.OWM('3ac4af53f33d1bb8a67da2676052a1a6')  # You MUST provide a valid API key
    L=[]
    with open(fold_csv, "r") as coord_base:
        coord=csv.reader(coord_base)
        for row in coord:
            L.append(row)
    A=str(L[0]).split(';') #split the string
    i=0
    long=[]
    lat=[]
    for i in range(len(L)):
        A=str(L[i]).split(';')
        long.append(A[0].strip('[\'')) # remove all part that we not need
        lat.append(A[1].strip(']\''))
    j=0
    for j in range(len(long)):
        long[j]=float(long[j]) # transform string in float
        lat[j]=float(lat[j])

    global Rain_amount
    Rain_amount=[] # create an empty list

    for cur in range(len(long)):
        loc=owm.weather_at_coords(lat[cur],long[cur]) # get the weather conditions


        w=loc.get_weather()
        rain=w.get_rain()
        if rain=={}: #if there is no rain it return {} but it's meen 0
            rainstr=0
        else:
            rainstr=float(str(rain)[6:-1]) #rain it's a str and we transform into a float
        Rain_amount.append(rainstr) # add to final list with all rain amount
    print(Rain_amount)
    sites_layer = arcpy.env.workspace + "/fishnet_w_coords"
    layerfields=["*"]
    arcpy.AddField_management("fishnet_w_coords","Add","FLOAT") # add an "Add" field to save the more recent data
    print("field created")
    fields=["*"]
    i=0
    with arcpy.da.UpdateCursor(sites_layer, fields) as cursor:
        for row in cursor:
            row[26]=Rain_amount[i] # add the weather condition to the right coordinate
            cursor.updateRow(row) #update to save the change
            i=i+1
    del cursor
    print("conditions added")
    arcpy.DeleteField_management("fishnet_w_coords", "Rain_amount_48h") # delete the oldest data from 48h ago
    print("rain 48h deleted")
    i=48
    for j in range(15):
        h=str(i-(j*3))
        hh=str(i-(j*3)-3)
        arcpy.AlterField_management("fishnet_w_coords","Rain_amount_"+hh+"h","Rain_amount_"+h+"h") #attribute the right name because of the offset (del 48h data), 45h->48h,42->45...
    arcpy.AlterField_management("fishnet_w_coords","Add_","Rain_amount_3h") #transform the add field with the 3h ago data with the right name
    print("Field names have been changed")
    print("start creating ")
    itt=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
    hour=3
    for rast in itt:
        nameraster=root + "/" + "IDW"+str(rast)
        if arcpy.Exists(nameraster):
            arcpy.Delete_management(nameraster)
            print("raster",nameraster,"deleted...")
        print("create the raster Rain_amount_"+str(hour)+"h")
        outIDW2 = arcpy.IDW_ga("fishnet_w_coords", "Rain_amount_"+str(hour)+"h","",nameraster) #create the IDW raster for each field
        hour=hour+3
    print("start final raster")
    global rastfin
    rastfin=0
    n=1/16
    for j in itt:
        rastfin=rastfin+(1+n-j*n)*arcpy.Raster(root + "/" + "IDW"+str(j)) #calcul the final raster with coefficient to add inportance to the recent data
    rastfin_name=root+"/"+"testrast"
    print(rastfin_name)
    if arcpy.Exists(rastfin_name):
        arcpy.Delete_management(rastfin_name)
    rastfin.save(rastfin_name)
    print("raster printing")
    print("end of the code")
    
    

In [30]:
def finalanswer(lat,long,city):
    owm = pyowm.OWM('3ac4af53f33d1bb8a67da2676052a1a6')
    name=lat+" "+long
    result_loc = arcpy.GetCellValue_management(rastfin, name) #get the pixel value of the localisation over the raster
    value=result_loc.getOutput(0)
    Valuechg=value.replace(",",".") #adapt the wrinting of the number
    Valuefloat=float(Valuechg) #transform to float
    fc = owm.three_hours_forecast(city)
    result_rain=fc.will_have_rain() #True there is Rain forcasted, false no rain
    if result_rain==True: #diff case
        if Valuefloat>1.0:
            print("The weather will be rainy at ",city," and the soil is humide:",Valuefloat,"mm, not need to add water")
        else:
            print("The weather will be rainy at ",city," and the soil is dry:",Valuefloat,"mm, keep an eye on futur whether")
    else:
        if Valuefloat>1.0:
            print("The weather doesn't anounce rain at ",city," but the soil is humide:",Valuefloat,"mm, so don't need to add water")
        else:
            print("The weather doesn't anounce rain at ",city," and the soil is dry:",Valuefloat,"mm, you may add water to your plant")

In [37]:
# générate weather conditions (run the cell after add your own folder)
folder_csv="C:\EsriTraining\RUDN2\Coord_project.csv" # with \
root_gdb = "C:/EsriTraining/RUDN2/dorianpro.gdb" # with / and keep dorianpro.gdb, do not create a new because there is the fishnet_w_coords file
arcpy.env.workspace = root_gdb # 
drap=arcpy.Exists(arcpy.env.workspace)
if drap==True:
    print("start executing code...")
    #rainupdate(folder_csv,root_gdb)
else:
    print("The workspace doesn't exist, check the root")
    
    

start executing code...


In [31]:
finalanswer("1,9","47,86","Olivet,FR")

The weather will be rainy at  Olivet,FR  and the soil is dry: 0.42467919 mm, keep an eye on futur whether


In [38]:
# test if the gdb has the file fishnet to add weather condition if it's false, try to check the root
sites_layer=arcpy.env.workspace + "/fishnet_w_coords"
arcpy.Exists(sites_layer)

True